In [31]:
# Imports
import pandas as pd
import seaborn as sns
import historicdutchweather
import pytz
from typing import Tuple
import pvlib
from datetime import datetime
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Business Understanding
We kijken naar zonnepaneeldata over een periode van net iets minder naar twee jaar. De metingen bestaan uit het wattage dat elke minuut door de panelen werd opgewekt. Hierbij is de data al omgezet naar watt per vierkante meter.

We zijn op zoek naar de efficientie van de panelen. Deze kan berekend worden door de theoretische hoeveelheid opbrengst te nemen en deze te verlagen met een bepaalde factor om de werkelijke meting te benaderen. Deze factor is de efficientie.

Om deze berekening te moeten doen, maken we gebruik van de pvlib library. Hierbij is het nodig om de helling van het zonnepaneel met het dak te weten (de tilt). En ook de richting van het paneel (de azimuth). De azimuth loopt van 0 graden noord, naar 90 graden oost, naar 180 graden zuid en dan naar 270 graden west.

# Data Understanding
## Stroomproductiedata ophalen

In [32]:
df = pd.read_csv('../../bijlagen/dataset.csv')
df['time'] = pd.to_datetime(df['time'])

In [33]:
df.head()

,time,production
0,2020-04-10 10:31:35+00:00,96.666667
1,2020-04-10 10:32:35+00:00,96.666667
2,2020-04-10 10:33:35+00:00,93.333333
3,2020-04-10 10:34:35+00:00,100.000000
4,2020-04-10 10:35:36+00:00,103.333333


In [34]:
df.tail()

,time,production
1266003,2022-09-06 17:32:35+00:00,0.000000
1266004,2022-09-06 17:33:35+00:00,3.333333
1266005,2022-09-06 17:34:35+00:00,0.000000
1266006,2022-09-06 17:35:35+00:00,0.000000
1266007,2022-09-06 17:36:35+00:00,3.333333


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1266008 entries, 0 to 1266007
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype              
---  ------      --------------    -----              
 0   time        1266008 non-null  datetime64[ns, UTC]
 1   production  1266008 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 19.3 MB


In [36]:
df.describe()

,production
count,1.266008e+06
mean,2.181287e+01
std,7.722842e+01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,2.000000e+01
max,2.206667e+03


## Weerdataset ophalen

In [37]:
import historicdutchweather

help(historicdutchweather.get_local_weather)

Help on function get_local_weather in module historicdutchweather.main:

get_local_weather(starttime: datetime.datetime, endtime: datetime.datetime, lat: float, lon: float, N_stations: int = 3, metrics: list = ['T', 'FH', 'DD', 'Q', 'DR', 'RH', 'U', 'N']) -> pandas.core.frame.DataFrame
    Get the localized hourly weather from the dutch KNMI website for a particular timeframe.

    Currently supports times starting as of 2010

    Timestamp represents the start of a particular hour. E.g. 14:00 represents 14:00-15:00

    Parameters
    ----------
    starttime : datetime object which represents the starting time
    endtime : datetime object which represents the ending time
    lat : target latitude
    lon : target longitude
    N_stations : number of stations to extrapolate the weather from, defaults to 3
    metrics : list of metrics to extrapolate



In [38]:
from historicdutchweather import get_local_weather
from datetime import datetime

# Stel de parameters in
starttime = datetime(2020, 4, 10, 0, 0, 0)  # Begin van de periode
endtime = datetime(2022, 9, 6, 23, 59, 59)  # Eind van de periode
latitude = 52.3676
longitude = 4.9041

# Verkrijg de weersgegevens
weather_data = get_local_weather(
    starttime=starttime,
    endtime=endtime,
    lat=latitude,
    lon=longitude,
    N_stations=3,
    metrics=['T', 'FH', 'DD', 'Q', 'DR', 'RH', 'U', 'N']
)

print(weather_data.head())

Model 2


c:\Users\MSI\Documents\Projecten\MachineLearning\venv\Lib\site-packages\historicdutchweather\main.py:67: DtypeWarning: Columns (15,16,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, comment="#", skiprows=30, skip_blank_lines=True, names=__headerline)
c:\Users\MSI\Documents\Projecten\MachineLearning\venv\Lib\site-packages\historicdutchweather\main.py:67: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, comment="#", skiprows=30, skip_blank_lines=True, names=__headerline)
100%|██████████| 21096/21096 [06:15<00:00, 56.17it/s]


                                  T         FH         DD              Q  \
2020-04-10 00:00:00+00:00  6.169539  10.447211  78.430644  1.114831e-316   
2020-04-10 01:00:00+00:00  5.787512   5.264597  10.819816  1.114831e-316   
2020-04-10 02:00:00+00:00  5.357216   4.264597  10.819816  1.114831e-316   
2020-04-10 03:00:00+00:00  5.608946   5.264597 -46.791011  1.114831e-316   
2020-04-10 04:00:00+00:00  5.052825  -0.918018 -41.006333  1.114831e-316   

                                      DR             RH          U    N  
2020-04-10 00:00:00+00:00  1.114831e-316  1.114831e-316  82.771855  NaN  
2020-04-10 01:00:00+00:00  1.114831e-316  1.114831e-316  79.986188  NaN  
2020-04-10 02:00:00+00:00  1.114831e-316  1.114831e-316  81.274519  NaN  
2020-04-10 03:00:00+00:00  1.114831e-316  1.114831e-316  83.289154  NaN  
2020-04-10 04:00:00+00:00  1.114831e-316  1.114831e-316  87.198875  NaN  


c:\Users\MSI\Documents\Projecten\MachineLearning\venv\Lib\site-packages\historicdutchweather\main.py:212: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  return pd.concat([df_template, df_local_weather])[metrics]
c:\Users\MSI\Documents\Projecten\MachineLearning\venv\Lib\site-packages\historicdutchweather\main.py:212: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df_template, df_local_weather])[metrics]


## Data Preparation

## Resampling naar xx minuten

In [39]:
# Resample de zonnepaneeldata naar elke 10 minuten (bijvoorbeeld)
df_resampled = df.resample('10T', on='time').mean()

C:\Users\MSI\AppData\Local\Temp\ipykernel_5028\1367633362.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_resampled = df.resample('10T', on='time').mean()


In [40]:
# Zet de huidige index terug naar een kolom met de naam 'time'
weather_data.reset_index(inplace=True)
weather_data.rename(columns={'index': 'time'}, inplace=True)  # Verander 'index' naar 'time'

In [41]:
weather_data.head()

,time,T,FH,DD,Q,DR,RH,U,N
0,2020-04-10 00:00:00+00:00,6.169539,10.447211,78.430644,1.114831e-316,1.114831e-316,1.114831e-316,82.771855,NaN
1,2020-04-10 01:00:00+00:00,5.787512,5.264597,10.819816,1.114831e-316,1.114831e-316,1.114831e-316,79.986188,NaN
2,2020-04-10 02:00:00+00:00,5.357216,4.264597,10.819816,1.114831e-316,1.114831e-316,1.114831e-316,81.274519,NaN
3,2020-04-10 03:00:00+00:00,5.608946,5.264597,-46.791011,1.114831e-316,1.114831e-316,1.114831e-316,83.289154,NaN
4,2020-04-10 04:00:00+00:00,5.052825,-0.918018,-41.006333,1.114831e-316,1.114831e-316,1.114831e-316,87.198875,NaN


## Weer en zonnepanelen combineren

In [42]:
# Stel de tijdstempel van de weerdata in als de index
weather_data['time'] = pd.to_datetime(weather_data['time'])
weather_data.set_index('time', inplace=True)

# Resample de weerdata naar elke 10 minuten
weather_data_resampled = weather_data.resample('10T').mean()

# Interpoleer de ontbrekende waarden
weather_data_resampled.interpolate(method='linear', inplace=True)

C:\Users\MSI\AppData\Local\Temp\ipykernel_5028\699764574.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  weather_data_resampled = weather_data.resample('10T').mean()
C:\Users\MSI\AppData\Local\Temp\ipykernel_5028\699764574.py:9: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  weather_data_resampled.interpolate(method='linear', inplace=True)


In [43]:
# Combineer de resampled data
combined_data = pd.merge(df_resampled, weather_data_resampled, left_index=True, right_index=True, how='inner')

In [44]:
print(type(combined_data.index))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [45]:
import pvlib

# Zorg ervoor dat 'times' een DatetimeIndex is
times = pd.DatetimeIndex(combined_data.index)

# Stel de locatie en andere parameters in
latitude = 52.3676
longitude = 4.9041
tilt = 30
azimuth = 180
location = pvlib.location.Location(latitude, longitude)
pv_system = pvlib.pvsystem.PVSystem(
    surface_tilt=tilt,
    surface_azimuth=azimuth
)

# Verkrijg de zonnestraling op basis van de weersgegevens
solar_position = location.get_solarposition(times)
clearsky = pvlib.clearsky.ineichen(times, latitude, longitude)

# Controleer de output van clearsky
print(clearsky.head())

# Zorg ervoor dat de output van clearsky de verwachte kolommen bevat
# Bijv. 'dni' en 'dhi' moeten aanwezig zijn
theoretical_power = pv_system.get_irradiance(
    solar_position['apparent_elevation'],
    solar_position['azimuth'],
    clearsky['dni'],
    clearsky['dhi']
)

# Voeg theoretische opbrengst toe aan je DataFrame
combined_data['theoretical_production'] = theoretical_power['poa_global']

# Bereken efficiëntie
combined_data['efficiency'] = combined_data['production'] / combined_data['theoretical_production']

TypeError: loop of ufunc does not support argument 0 of type Timestamp which has no callable radians method

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.lineplot(data=combined_data, x=combined_data.index, y='efficiency')
plt.title('Efficiëntie van de Zonnepanelen Over Tijd')
plt.xlabel('Tijd')
plt.ylabel('Efficiëntie')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Filter op zonnige dagen

# Modeling

In [ ]:
# Gegeven:
def model_zonnestraling(tilt:float, azimuth:float, tijdsreeks:pd.core.indexes.datetimes.DatetimeIndex, lat:float, lon:float) -> pd.DataFrame:
    """Berekent de hoeveelheid stroom die een paneel maximaal produceert met 100% efficientie"""

    zonne_positie_aan_hemel = pvlib.solarposition.get_solarposition(tijdsreeks, lat, lon)

    #locatie = pvlib.location.Location(latitude=lat, longitude=lon, name='Plaatsnaam', tz=pytz.timezone(r'Europe/Amsterdam'))
    locatie = pvlib.location.Location(latitude=lat, longitude=lon, name='Plaatsnaam', tz=pytz.timezone(r'UTC'))
    heldere_hemel = locatie.get_clearsky(tijdsreeks)

    instraling = pvlib.irradiance.get_total_irradiance(tilt,
                                                       azimuth,
                                                       zonne_positie_aan_hemel['zenith'],
                                                       zonne_positie_aan_hemel['azimuth'],
                                                       heldere_hemel.dni,
                                                       heldere_hemel.ghi,
                                                       heldere_hemel.dhi)

    return instraling['poa_global'].to_frame().rename(columns={'poa_global':'zonlicht'})

In [ ]:
# Aanvullen
def model_paneel(tilt:float, azimuth:float, efficientie:float, tijdsreeks:pd.core.indexes.datetimes.DatetimeIndex, lat:float, lon:float) -> pd.DataFrame:
    """Berekent de hoeveelheid geabsorbeerd zonlicht op een paneel op basis van de efficientie"""

    return ... # Aanvullen

In [ ]:
# Aanvullen
def loss(params:Tuple[float,float,float], df_waarneming:pd.DataFrame, lat:float, lon:float) -> float:

    (tilt, azimuth, efficientie) = params

    tijdsreeks = df_waarneming.index

    df_model = model_paneel(...) # Aanvullen

    return # Aanvullen


In [ ]:
# Eerste inschatting
tilt = 35
azimuth = 180
efficientie = 0.2

In [ ]:
result = minimize(...) # Aanvullen
x = result['x']

## Evaluation

In [ ]:
print("Tilt:        {0:.1f}°".format(x[0]))
print("Azimuth:     {0:.1f}°".format(x[1]))
print("Efficientie: {0:.1f}%".format(x[2]*100))